In [ ]:
!pip install tqdm
!pip install datasets
import json
from tqdm import tqdm
from datasets import Dataset as ds
import random

In [2]:
file_path = r"/content/drive/MyDrive/IRE/project/train_chaii_hi_1.json"
train_file_path = r"/content/drive/MyDrive/IRE/project/train.json"
test_file_path = r"/content/drive/MyDrive/IRE/project/test.json"

In [12]:
import json
import random

# Specify the path to your original JSON file
json_file_path = file_path

# Load the data from the original JSON file
try:
    with open(json_file_path, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
    print("Original JSON file loaded successfully.")
except FileNotFoundError:
    print(f"Error: JSON file '{json_file_path}' not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Shuffle the data to ensure randomness
random.seed(42)  # For reproducibility, you can choose any seed value
random.shuffle(data)

# Calculate the index for splitting
split_index = int(0.2 * len(data))  # 20% for testing, 80% for training

# Split the data into test and train sets
test_data = data[:split_index]
train_data = data[split_index:]

# Define paths for the train and test JSON files
train_json_path = "train.json"
test_json_path = "test.json"

# Write the data from train_data to train.json
with open(train_json_path, "w", encoding="utf-8") as train_json_file:
    json.dump(train_data, train_json_file, ensure_ascii=False, indent=4)

# Write the data from test_data to test.json
with open(test_json_path, "w", encoding="utf-8") as test_json_file:
    json.dump(test_data, test_json_file, ensure_ascii=False, indent=4)

print(f"Data split and saved to {train_json_path} and {test_json_path}.")

Original JSON file loaded successfully.
Data split and saved to train.json and test.json.


In [3]:
# Function to load data from a file into the dataset
def load_data(file):

  # Specify the path to your JSON file
  json_file_path = file
  # Initialize an empty dictionary
  data = {}

  # Load the JSON data into the dictionary
  try:
      with open(json_file_path, "r") as json_file:
          data = json.load(json_file)
      print("JSON file loaded successfully into a dictionary.")
  except FileNotFoundError:
      print(f"Error: JSON file '{json_file_path}' not found.")
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")

  return data

train_dict = load_data(train_file_path)
test_dict = load_data(test_file_path)
print("len(train_dict): ", len(train_dict))
print("len(test_dict): ", len(test_dict))

JSON file loaded successfully into a dictionary.
JSON file loaded successfully into a dictionary.
len(train_dict):  596
len(test_dict):  149


In [8]:
chai_dataset = {
  "train": {"question": [], "positivie_context": [], "negative_context": []},
  "test": {"question": [], "positivie_context": [], "negative_context": []},
}

def get_data(split_type, dict_type, chai_dataset, k):

  question_list = []
  positive_contest_list = []
  negative_contest_list = []
  random_seed = 42
  random.seed(random_seed)
  for i in dict_type:
    question_list.append(i['question'])
    positive_contest_list.append(i["positive_ctxs"][0]['text'])

  for i in range(len(positive_contest_list)):
    list1 =[]
    for j in range(len(positive_contest_list)):
      if(i!=j):
        list1.append(positive_contest_list[j])
    list1 = random.choices(list1, k=k)
    negative_contest_list.append(list1[:])

  for i in range(len(question_list)):
    chai_dataset[split_type]["question"] = question_list
    chai_dataset[split_type]["positivie_context"] = positive_contest_list
    chai_dataset[split_type]["negative_context"] = negative_contest_list

#build train
get_data("train", train_dict, chai_dataset, 10)

#build test
get_data("test", test_dict, chai_dataset, 10)



In [7]:
# print(len(chai_dataset["train"]["question"]))
# print(len(chai_dataset["train"]["positivie_context"]))
# print(len(chai_dataset["train"]["negative_context"]))
# print(len(chai_dataset["train"]["negative_context"][0]))
# # print(chai_dataset["train"]["question"][0])
# # print(chai_dataset["train"]["positivie_context"][0])
# # print(chai_dataset["train"]["negative_context"][0])
# # print(chai_dataset["train"]["negative_context"][0][0])
# print("###################################################")
# print(len(chai_dataset["test"]["question"]))
# print(len(chai_dataset["test"]["positivie_context"]))
# print(len(chai_dataset["test"]["negative_context"]))
# print(len(chai_dataset["test"]["negative_context"][0]))
# # print(chai_dataset["test"]["question"][0])
# # print(chai_dataset["test"]["positivie_context"][0])
# # print(chai_dataset["test"]["negative_context"][0])
# # print(chai_dataset["test"]["negative_context"][0][0])

596
596
596
10
###################################################
149
149
149
10


In [9]:
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset

class chaiDataset(Dataset):
  def __init__(self, dataset_object, split_type):
    #convert list to pandas datafraame
    #create new df
    # split_type = 'train'

    self.df = pd.DataFrame({'question':dataset_object[split_type]['question'],'positivie_context':dataset_object[split_type]['positivie_context'],'negative_context':dataset_object[split_type]['negative_context']})
    # print ("df:",self.df)

    # Get question columns
    self.question = self.df["question"]
    # Get positive_context columns
    self.positivie_context = self.df["positivie_context"]
    # Get the negative_context columns
    self.negative_context = self.df["negative_context"]

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    question = self.question[index]
    positive_context = self.positivie_context[index]
    negative_context = self.negative_context[index]

    return question, positive_context, negative_context

def get_loader(
    dataset_object,
    split_type,
    batch_size=1,
    shuffle=True,
):
  dataset = chaiDataset(chai_dataset, split_type)

  loader = DataLoader(
      dataset=dataset,
      batch_size=batch_size,
      shuffle=shuffle,
  )

  return loader, dataset


In [10]:
test_dataset_object = chai_dataset["test"]
train_dataset_object = chai_dataset["train"]
test_split_type = "test"
train_split_type = "train"
batch_size = 1
test_dataloader, test_dataset = get_loader(test_dataset_object, test_split_type, batch_size)
train_dataloader, train_dataset = get_loader(train_dataset_object, train_split_type, batch_size)

In [23]:
# test_dataset.positivie_context
# test_dataset.question
# test_dataset.negative_context

# train_dataset.positivie_context
# train_dataset.question
# train_dataset.negative_context

'मानव शरीर रचना विज्ञान में, आंत (या अंतड़ी) आहार नली का हिस्सा होती है जो पेट से गुदा तक फैली होती है, तथा मनुष्य और अन्य स्तनधारियों में, यह दो भागों में, छोटी आंत और बड़ी आंत के रूप में होती है। मनुष्यों में, छोटी आंत को आगे फिर पाचनांत्र, मध्यांत्र और क्षुद्रांत्र में विभाजित किया गया है, जबकि बड़ी आंत को अंधात्र और बृहदान्त्र में विभाजित किया गया है।[1]\nसंरचना और कार्य संरचना और कार्य दोनों को संपूर्ण शारीरिक रचना विज्ञान के रूप में और अतिसूक्ष्म स्तर के रूप में वर्णित किया जा सकता है। आंत्र क्षेत्र को मोटे तौर पर दो अलग-अलग भागों, छोटी और बड़ी आंत में विभाजित किया जा सकता है। एक वयस्क व्यक्ति में भूरे- बैंगनी रंग की छोटी आंत का व्यास लगभग 35 मिलीमीटर (1.5 इंच) और औसत लंबाई 6 से 7 मीटर (20-23 फुट) होती है। गहरे लाल रंग की बड़ी आंत छोटी और अपेक्षाकृत मोटी होती है, जिसकी लंबाई औसत रूप से लगभग 1.5 मीटर (5 फुट) होती है।[2]\nआकार और उम्र के अनुसार व्यक्तियों में अलग अलग आकार की आंतें होंगी.\nल्यूमेन (अवकाशिका) वह गुहा है जहां से पचा हुआ भोजन गुजरता है तथा जहां से पोषक तत्व अवशोषित होत

In [24]:
for q, p, n in test_dataloader:
  print("q: ",q)
  print("p: ",p)
  print("n: ",n)
  break

q:  ('हिन्दू धर्म में रंग के त्यौहार का नाम क्या है?',)
p:  ("होली (Holi) वसंत ऋतु में मनाया जाने वाला एक महत्वपूर्ण भारतीय और नेपाली लोगों का त्यौहार है। यह पर्व हिंदू पंचांग के अनुसार फाल्गुन मास की पूर्णिमा को मनाया जाता है। रंगों का त्यौहार कहा जाने वाला यह पर्व पारंपरिक रूप से दो दिन मनाया जाता है। यह प्रमुखता से भारत तथा नेपाल में मनाया जाता है। यह त्यौहार कई अन्य देशों जिनमें अल्पसंख्यक हिन्दू लोग रहते हैं वहाँ भी धूम-धाम के साथ मनाया जाता है।[1] पहले दिन को होलिका जलायी जाती है, जिसे होलिका दहन भी कहते हैं। दूसरे दिन, जिसे प्रमुखतः धुलेंडी व धुरड्डी, धुरखेल या धूलिवंदन इसके अन्य नाम हैं, लोग एक दूसरे पर रंग, अबीर-गुलाल इत्यादि फेंकते हैं, ढोल बजा कर होली के गीत गाये जाते हैं और घर-घर जा कर लोगों को रंग लगाया जाता है। ऐसा माना जाता है कि होली के दिन लोग पुरानी कटुता को भूल कर गले मिलते हैं और फिर से दोस्त बन जाते हैं। एक दूसरे को रंगने और गाने-बजाने का दौर दोपहर तक चलता है। इसके बाद स्नान कर के विश्राम करने के बाद नए कपड़े पहन कर शाम को लोग एक दूसरे के घर मिलने जाते हैं, गले मिल